# Assemblage liste de données

## Ajout d'un colonne & Fusion des fichier

In [1]:
import os
import pandas as pd

In [2]:
dossier = "CSV"

In [3]:
# Test a enlever
noms_fichiers_formatés = []

for fichier in os.listdir(dossier):
    if fichier.endswith(".csv"):
        # Suppression de l'extension
        nom_sans_extension = fichier.replace('.csv', '')
        # Rajout d'une majuscule au début du nom
        nom_formaté = nom_sans_extension[0].upper() + nom_sans_extension[1:]
        noms_fichiers_formatés.append(nom_formaté)
noms_fichiers_formatés

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'Base'

In [ ]:
donnees_totales = []

In [ ]:
for fichier in os.listdir(dossier):
    if fichier.endswith(".csv"):
        chemin_fichier = os.path.join(dossier, fichier)
        # Lecture du fichier CSV
        df = pd.read_csv(chemin_fichier)
        # Ajout de la colonne "file" avec le nom du fichier (sans extension .csv)
        nom_sans_extension = fichier.replace('.csv', '')
        df['Montagne'] = nom_sans_extension[0].upper() + nom_sans_extension[1:]
        # Ajout du DataFrame à la liste
        donnees_totales.append(df)
 

In [ ]:
# Concaténation de tous les DataFrames en un seul
donnees_combinees = pd.concat(donnees_totales, ignore_index=True)
# Enregistrement des données combinées dans un fichier CSV
donnees_combinees.to_csv('Donnees_Combinees.csv', index=False)

## Déplacement de colonne

In [ ]:
# Chemin vers le fichier CSV d'entrée et de sortie
chemin_fichier_entree = "Donnees_Combinees.csv"
chemin_fichier_sortie = "Donnees_Modifiees.csv"

In [ ]:
# Lire le fichier CSV avec pandas
donnees = pd.read_csv(chemin_fichier_entree)

In [ ]:
# Réorganiser les colonnes, en mettant 'Montagne' devant 'Date'
colonnes = ['Montagne', 'Date', 'Name', 'Nationality', 'Cause of death']

In [ ]:
# Réorganiser les colonnes du DataFrame
donnees = donnees[colonnes]

In [ ]:
# Écrire le DataFrame modifié dans un nouveau fichier CSV
donnees.to_csv(chemin_fichier_sortie, index=False)

## Date changement de format

In [ ]:
from datetime import datetime

In [ ]:
# Fonction pour convertir la date en format jour/mois/année en nombre
def convertir_date(date_str):
    # Liste des formats possibles de date
    formats = ['%d %B %Y', '%B %d, %Y', '%Y, %dth %B', '%B %Y', '%Y']

    # Essayer de convertir la date avec différents formats
    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            # Si la date ne contient pas de jour, retourner seulement le mois et l'année
            if '%d' not in fmt:
                return date_obj.strftime('%m/%Y')
            else:
                return date_obj.strftime('%d/%m/%Y')
        except ValueError:
            pass
    # Si aucun format ne correspond, retourner la date telle quelle
    return date_str

In [ ]:
# Chemin vers le fichier CSV d'entrée et de sortie
chemin_fichier_entree = "Donnees_Combinees.csv"
chemin_fichier_sortie = "Donnees_Modifiees.csv"

# Lire le fichier CSV avec pandas
donnees = pd.read_csv(chemin_fichier_entree)

# Appliquer la fonction de conversion sur la colonne 'Date'
donnees['Date'] = donnees['Date'].apply(convertir_date)

# Réorganiser les colonnes, en mettant 'Montagne' devant 'Date'
colonnes = ['Montagne', 'Date', 'Name', 'Nationality', 'Cause of death']

# Réorganiser les colonnes du DataFrame
donnees = donnees[colonnes]

# Écrire le DataFrame modifié dans un nouveau fichier CSV
donnees.to_csv(chemin_fichier_sortie, index=False)